## 作業目標
* 嘗試比較用 color histogram 和 HOG 特徵來訓練的 SVM 分類器在 cifar10 training 和 testing data 上準確度的差別

In [1]:
import os
from tensorflow import keras
os.environ["CUDA_VISIBLE_DEVICES"] = "" # 使用 CPU

import numpy as np
import cv2 # 載入 cv2 套件
import matplotlib.pyplot as plt

train, test = keras.datasets.cifar10.load_data()

In [2]:
x_train, y_train = train
x_test, y_test = test
y_train = y_train.astype(int)
y_test = y_test.astype(int)

#### 產生直方圖特徵的訓練資料

In [3]:
x_train_histogram = []
x_test_histogram = []

# 對於所有訓練資料
for i in range(len(x_train)):
    chans = cv2.split(x_train[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    # 把計算的直方圖特徵收集起來
    x_train_histogram.append(hist_feature)

# 對於所有測試資料也做一樣的處理
for i in range(len(x_test)):
    chans = cv2.split(x_test[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    x_test_histogram.append(hist_feature)

x_train_histogram = np.array(x_train_histogram)
x_test_histogram = np.array(x_test_histogram)


In [4]:
print(x_train_histogram.shape)
print(x_test_histogram.shape)
print(x_train_histogram[0])

(50000, 48)
(10000, 48)
[  1.   4.   9.  19.  28.  36.  80. 162. 286. 158.  65.  52.  41.  36.
  28.  19.   6.  17.  35.  57. 111. 277. 204.  97.  53.  49.  38.  24.
  17.  18.  14.   7.  34. 108. 238. 296. 127.  63.  49.  21.  22.  14.
  16.   9.  13.   9.   5.   0.]


#### 產生 HOG 特徵的訓練資料
* HOG 特徵通過計算和統計圖像局部區域的梯度方向直方圖來構建特徵，具體細節不在我們涵蓋的範圍裡面，有興趣的同學請參考[補充資料](https://www.cnblogs.com/zyly/p/9651261.html)哦

In [5]:
# SZ=20
bin_n = 16 # Number of bins

def hog(img):
    # RGB轉換成灰階
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # 使用 Sobel 運算元取出圖像邊界
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    
    # 計算出圖像的大小與角度
    mag, ang = cv2.cartToPolar(gx, gy)
    
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist.astype(np.float32)

x_train_hog = np.array([hog(x) for x in x_train])
x_test_hog = np.array([hog(x) for x in x_test])

In [6]:
print(x_train_hog.shape)
print(x_test_hog.shape)
print(x_train_hog[0])

(50000, 64)
(10000, 64)
[ 938.37823  2090.8708    752.14545   613.8052   1423.8641    253.43987
  233.64297    74.59453   275.92752    61.03782   193.41498   598.6044
  251.85165    43.266613  142.39381   522.83765  1068.8157   7444.274
  544.7197    327.72504  2181.5527   2147.2158   1204.5164   1618.63
 2170.1375   1877.6937    714.6225    861.39636  2205.8662   1573.037
  967.32666   770.9872   1536.555     697.71326   906.7769    690.0921
 2358.1633    777.9613   1767.8911    664.54065  1327.0061   1251.5668
 2227.4338   2628.579    3016.9563   1033.8795   1209.8636    923.1289
 4726.9653   5058.291    3775.839    7533.1274   7835.028    3041.563
 3276.0295   5005.9893   8706.14     4403.464    3899.6396   6273.0776
 4921.891    3419.6157   4027.9946   2642.0925  ]


#### SVM model
* SVM 是機器學習中一個經典的分類算法，具體細節有興趣可以參考 [該知乎上的解釋](https://www.zhihu.com/question/21094489)，我們這裡直接調用 opencv 中實現好的函數

#### 用 histogram 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [7]:
SVM_hist = cv2.ml.SVM_create()
SVM_hist.setKernel(cv2.ml.SVM_LINEAR)
SVM_hist.setGamma(5.383)
SVM_hist.setType(cv2.ml.SVM_C_SVC)
SVM_hist.setC(2.67)

#training
SVM_hist.train(x_train_histogram, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hist_train = SVM_hist.predict(x_train_histogram)
_, y_hist_test = SVM_hist.predict(x_test_histogram)

In [8]:
print(y_hist_train.shape)
y_hist_train

(50000, 1)


array([[4.],
       [8.],
       [1.],
       ...,
       [9.],
       [0.],
       [0.]], dtype=float32)

In [9]:
from sklearn.metrics import accuracy_score

accu = accuracy_score(y_train, y_hist_train)
print("Train Accuracy: %.4f" % (accu) )
accu = accuracy_score(y_test, y_hist_test)
print("Test Accuracy: %.4f" % (accu) )

Train Accuracy: 0.1485
Test Accuracy: 0.1448


#### 用 HOG 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [10]:
SVM_hog = cv2.ml.SVM_create()
SVM_hog.setKernel(cv2.ml.SVM_LINEAR)
SVM_hog.setGamma(5.383)
SVM_hog.setType(cv2.ml.SVM_C_SVC)
SVM_hog.setC(2.67)

#training
SVM_hog.train(x_train_hog, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hog_train = SVM_hog.predict(x_train_hog)
_, y_hog_test = SVM_hog.predict(x_test_hog)

In [11]:
print(y_hog_train.shape)
y_hog_train

(50000, 1)


array([[1.],
       [7.],
       [5.],
       ...,
       [1.],
       [9.],
       [8.]], dtype=float32)

In [12]:
from sklearn.metrics import accuracy_score

accu = accuracy_score(y_train, y_hog_train)
print("Train Accuracy: %.4f" % (accu) )
accu = accuracy_score(y_test, y_hog_test)
print("Test Accuracy: %.4f" % (accu) )

Train Accuracy: 0.1928
Test Accuracy: 0.1905
